In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
id_gpu = '3'
n_users = 105
os.environ['CUDA_VISIBLE_DEVICES'] = id_gpu

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from datasets import load_dataset

dataset = load_dataset("LeoTungAnh/kdd210_hourly")
dataset

DatasetDict({
    train: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    validation: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
    test: Dataset({
        features: ['start', 'feat_static_cat', 'feat_dynamic_real', 'item_id', 'target'],
        num_rows: 210
    })
})

In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
len(test_dataset[0]['target'])

10898

In [8]:
type(test_dataset)

datasets.arrow_dataset.Dataset

In [9]:
import pandas as pd

# Create a list of dictionaries, where each dictionary represents a row in the DataFrame
df = pd.DataFrame()

for row in test_dataset:
  df[row['item_id']] = row['target']

df['date'] = '2015-01-01 00:00:00'
df = df.set_index("date")
df.head()

/tmp/ipykernel_4085623/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[row['item_id']] = row['target']
/tmp/ipykernel_4085623/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[row['item_id']] = row['target']
/tmp/ipykernel_4085623/2019493983.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10,...,T201,T202,T203,T204,T205,T206,T207,T208,T209,T210
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,1.468122,1.538769,-0.030617,-0.781501,-0.802695,-0.772417,-0.378809,-0.237514,-0.393948,-0.810264,...,-0.131542,-0.785538,-0.807741,-0.777464,1.493353,1.654833,-0.025571,-0.776959,-0.777464,-0.792602
2015-01-01 00:00:00,1.316855,1.449373,-0.079674,-0.773862,-0.798327,-0.767746,-0.512904,0.000000,-0.507808,-0.803933,...,-0.140836,-0.775391,-0.798327,-0.772842,1.444276,1.724601,-0.023609,-0.804443,-0.767746,-0.783036
2015-01-01 00:00:00,1.261700,1.636722,-0.061296,-0.759776,-0.780089,-0.754046,-0.639456,-0.582161,-0.597787,-0.792069,...,-0.134217,-0.759776,-0.780089,-0.754046,1.501297,1.772147,-0.009210,-0.791548,-0.748838,-0.769672
2015-01-01 00:00:00,1.487590,1.831615,-0.022891,-0.730291,-0.753942,-0.721690,-0.646435,-0.560429,-0.555053,-0.765768,...,-0.114272,-0.733516,-0.753942,-0.732441,0.525398,0.000000,-0.270158,-0.744267,-0.678687,-0.748567
2015-01-01 00:00:00,1.842409,2.225288,0.150884,-0.702879,-0.723452,-0.694879,-0.632018,-0.500582,-0.454865,-0.741738,...,-0.066271,-0.707451,-0.729166,-0.717737,-0.672020,0.000000,-0.569157,-0.743453,-0.557728,-0.723452


In [10]:
# # Save the DataFrame to a CSV file
# df.to_csv('/home/qula0496/quan/Nonstationary_Transformers/dataset/kdd/kdd.csv', index_label='date')

In [11]:
df.shape

(10898, 210)

In [12]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(df.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = id_gpu
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path = './dataset/kdd/'
args.data_path = 'kdd.csv'
args.model_id = 'kdd_96_96'
args.data_path

'kdd.csv'

In [13]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [14]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, df.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, df.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [15]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
            

In [16]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 30
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.1
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on KDD dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on KDD dataset",
    # Hyperparams
    config={
        "dataset": "KDD210",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 210,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [17]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1 cost time: 127.75732350349426
Epoch: 2 cost time: 126.65289282798767
Epoch: 3 cost time: 131.0310080051422
Epoch: 4 cost time: 136.52307391166687
Epoch: 5 cost time: 134.7539291381836
Epoch: 1 cost time: 135.56364274024963
Epoch: 2 cost time: 136.31902933120728
Epoch: 3 cost time: 135.26492857933044
Epoch: 4 cost time: 137.78272485733032
Epoch: 5 cost time: 136.92661213874817
Epoch: 1 cost time: 136.4463129043579
Epoch: 2 cost time: 135.60170030593872
Epoch: 3 cost time: 136.05807995796204
Epoch: 4 cost time: 135.1650106906891
Epoch: 5 cost time: 135.62923002243042
Epoch: 1 cost time: 136.30268549919128
Epoch: 2 cost time: 135.67643237113953
Epoch: 3 cost time: 136.75578618049622
Epoch: 4 cost time: 136.73443269729614
Epoch: 5 cost time: 136.1241865158081
Epoch: 1 cost time: 136.76477241516113
Epoch: 2 cost time: 135.40033102035522
Epoch: 3 cost time: 135.91369557380676
Epoch: 4 cost time: 135.1398365497589
Epoch: 5 cost time: 136.13630962371826
Epoch: 1 cost time: 135.8289608

Progress:   3%|▎         | 1/30 [2:09:54<62:47:10, 7794.17s/it]

Epoch: 1 cost time: 137.73046231269836
Epoch: 2 cost time: 137.0225706100464
Epoch: 3 cost time: 137.15936255455017
Epoch: 4 cost time: 135.99600744247437
Epoch: 5 cost time: 136.8028063774109
Epoch: 1 cost time: 138.4730145931244
Epoch: 2 cost time: 137.115332365036
Epoch: 3 cost time: 138.31011819839478
Epoch: 4 cost time: 137.89063215255737
Epoch: 5 cost time: 137.89288997650146
Epoch: 1 cost time: 137.6614851951599
Epoch: 2 cost time: 136.38271808624268
Epoch: 3 cost time: 136.74871635437012
Epoch: 4 cost time: 136.95905709266663
Epoch: 5 cost time: 136.62680888175964
Epoch: 1 cost time: 138.02066135406494
Epoch: 2 cost time: 137.5133535861969
Epoch: 3 cost time: 136.73435807228088
Epoch: 4 cost time: 137.85232758522034
Epoch: 5 cost time: 136.138601064682
Epoch: 1 cost time: 137.79493713378906
Epoch: 2 cost time: 136.18036317825317
Epoch: 3 cost time: 137.15103101730347
Epoch: 4 cost time: 137.44864296913147
Epoch: 5 cost time: 136.98805141448975
Epoch: 1 cost time: 137.6068782806

Progress:   7%|▋         | 2/30 [4:21:05<60:58:29, 7839.61s/it]

...

Progress:  93%|█████████▎| 28/30 [37:19:17<1:18:05, 2342.93s/it]

Epoch: 1 cost time: 34.44207310676575
Epoch: 2 cost time: 34.33778762817383
Epoch: 3 cost time: 34.61681509017944
Epoch: 4 cost time: 34.9033145904541
Epoch: 5 cost time: 34.35574388504028
Epoch: 1 cost time: 35.12830185890198
Epoch: 2 cost time: 35.1842474937439
Epoch: 3 cost time: 32.6386194229126
Epoch: 4 cost time: 35.00375485420227
Epoch: 5 cost time: 34.77653932571411
Epoch: 1 cost time: 36.789286851882935
Epoch: 2 cost time: 35.07015252113342
Epoch: 3 cost time: 40.64100766181946
Epoch: 4 cost time: 39.21177816390991
Epoch: 5 cost time: 40.42116451263428
Epoch: 1 cost time: 39.349912881851196
Epoch: 2 cost time: 39.43601751327515
Epoch: 3 cost time: 38.60093808174133
Epoch: 4 cost time: 34.4305157661438
Epoch: 5 cost time: 35.16489577293396
Epoch: 1 cost time: 34.97809171676636
Epoch: 2 cost time: 34.19050216674805
Epoch: 3 cost time: 33.61496877670288
Epoch: 4 cost time: 34.073261737823486
Epoch: 5 cost time: 37.68693995475769
Epoch: 1 cost time: 38.0388867855072
Epoch: 2 cost 

Progress:  97%|█████████▋| 29/30 [37:54:47<37:58, 2278.88s/it]  

Epoch: 1 cost time: 34.177650451660156
Epoch: 2 cost time: 35.5270562171936
Epoch: 3 cost time: 35.162850856781006
Epoch: 4 cost time: 35.99535512924194
Epoch: 5 cost time: 35.10716915130615
Epoch: 1 cost time: 34.04874849319458
Epoch: 2 cost time: 34.56029677391052
Epoch: 3 cost time: 35.29705190658569
Epoch: 4 cost time: 34.04697132110596
Epoch: 5 cost time: 35.05713510513306
Epoch: 1 cost time: 33.975332260131836
Epoch: 2 cost time: 36.862690687179565
Epoch: 3 cost time: 34.9051468372345
Epoch: 4 cost time: 34.52526903152466
Epoch: 5 cost time: 33.897449016571045
Epoch: 1 cost time: 34.62004208564758
Epoch: 2 cost time: 35.58899474143982
Epoch: 3 cost time: 33.98547053337097
Epoch: 4 cost time: 33.94120669364929
Epoch: 5 cost time: 33.85749816894531
Epoch: 1 cost time: 35.11144185066223
Epoch: 2 cost time: 35.123332023620605
Epoch: 3 cost time: 33.92858934402466
Epoch: 4 cost time: 35.130332708358765
Epoch: 5 cost time: 35.64695692062378
Epoch: 1 cost time: 35.85206699371338
Epoch: 

Progress: 100%|██████████| 30/30 [38:29:05<00:00, 4618.19s/it]
